In [92]:
import scipy
import numpy as np
from itertools import product

In [121]:
shipping_cost = np.array([[1,3],[3,2],[2,2]]).reshape(3,2)
production_cost = np.array([6,5]).reshape(1,2)
given_demand = np.array([[50,60,75],[75,90,100],[60,75,90]]).reshape(3,3)
produce_limit = np.array([250,250]).reshape(2,1)
senario = 2700
senario_prob = (1/senario)*(np.ones((senario,1)))

In [122]:
products = (list(product(range(3),repeat=3)))
senarios = [[given_demand[i][j]  for i,j in enumerate(pro)] for pro in products]

In [123]:
destruct_senarios = np.random.binomial(1,0.3,(100,3,2))
destruct_units = np.where(destruct_senarios == 1, 0.5, 1)

In [124]:
# Create shortfall matrix 
#shortfall = -1*np.eye((100*27*3,100*27*3))
shortfall = -1*np.eye((100*27*3))

#create empty numpy array
array = np.array([])
demand_array = np.array([])

for demand in senarios:
    for destruct in destruct_units:
        transport_mat = np.zeros((3,6))

        for ind,i in enumerate(demand):
            transport_mat[ind,2*ind] = -1*destruct[ind][0]
            transport_mat[ind,2*ind+1] = -1*destruct[ind][1]
            demand_array =  np.append(demand_array,-1*i)

        array = np.append(array,transport_mat)
array = array.reshape(100*27*3,6)

In [125]:
obj =[]
bound = []



In [126]:
for i in production_cost:
    obj.extend(i)

for i in produce_limit:
    bound.append([0,i[0]])

In [127]:
bound

[[0, np.int64(250)], [0, np.int64(250)]]

In [128]:
obj

[np.int64(6), np.int64(5)]

In [101]:
for i in shipping_cost:
    for j in i:
        obj.extend([j])
        bound.append([0,250])

for i in range(3*27*100):
    obj.extend([40/2700])
    bound.append([0,float("inf")])

In [102]:
# Create 2,2 diagonal matrix with ones
C = -1*np.eye(2)

B=  np.repeat(np.eye(2),3,axis=1)

D = np.zeros((2,100*27*3))

In [103]:
C

array([[-1., -0.],
       [-0., -1.]])

In [104]:
B

array([[1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1.]])

In [105]:
F = np.concatenate((C,B,D),axis=1)

In [106]:
G = np.zeros((2,1))

In [107]:
c = -1*np.eye((2))

In [108]:
shipping_cost[0]

array([1, 3])

In [109]:
# A_eq = np.zeros((2,len(obj)))
# b_eq = np.zeros((2,1))

# A_eq[0][0] = 1
# A_eq[0][1] = 1

# b_eq[0] = 250
# b_eq[1] = 250

In [110]:
zeros = np.zeros((100*27*3,2))

In [111]:
A_ub = np.concatenate((zeros,array,shortfall),axis=1)

In [112]:
demand_array = demand_array.reshape(100*27*3,1)

In [113]:
A_ub = np.concatenate((A_ub,F),axis=0)
demand_array = np.concatenate((demand_array,G),axis=0)

In [138]:
print(list(A_ub[5][0:9]))

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(-1.0), np.float64(-1.0), np.float64(-0.0)]


In [115]:
opt = scipy.optimize.linprog(c=obj, A_ub=A_ub, b_ub=demand_array,bounds=bound,method="highs")

In [117]:
opt

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 2718.666666666642
              x: [ 1.133e+02  1.833e+02 ...  4.167e+01  0.000e+00]
            nit: 3745
          lower:  residual: [ 1.133e+02  1.833e+02 ...  4.167e+01
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              1.481e-02]
          upper:  residual: [ 1.367e+02  6.667e+01 ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 3.000e+01  4.167e+01 ...  0.000e+00
                              0.000e+00]
                 marginals: [-0.000e+00 -0.000e+00 ... -6.000e+00
                             -5.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mi

In [215]:
bound[-10:]

[(0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf)]

In [120]:
opt.x[0:10]

array([113.33333333, 183.33333333,  40.        ,  40.        ,
        33.33333333,  83.33333333,  50.        ,  50.        ,
         0.        ,   0.        ])

In [20]:
opt.x

array([-0., -0.,  0., ...,  0.,  0.,  0.])

In [91]:
opt.x[0:2]

array([-0., -0.])

In [21]:
opt

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.037037037037037035
              x: [-0.000e+00 -0.000e+00 ...  0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [-0.000e+00 -0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [ 2.500e+02  2.500e+02 ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 5.000e+01  2.500e+01 ...  0.000e+00
                              0.000e+00]
                 marginals: [-0.000e+00 -0.000e+00 ... -6.000e+00
                             -5.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mi